In [1]:
from qboard import Solver
import numpy as np

from config import PARAMS

In [2]:
s = Solver(mode="remote:gurobi", params=PARAMS)

In [3]:
Q = np.random.randn(5,5)

spins, energy = s.solve_qubo(Q, timeout=30)
spins, energy

[2021-10-23 15:49:08] Solver remote:gurobi started
[2021-10-23 15:49:08] Server status - 0 active tasks
[2021-10-23 15:49:08] Start matrix upload
[2021-10-23 15:49:08] Upload completed, start solving
[2021-10-23 15:49:09] Found solution 0.000000 
[2021-10-23 15:49:09] Found solution -3.745649 
[2021-10-23 15:49:09] Found solution -7.079519 mipgap=1.42751729269445
[2021-10-23 15:49:09] Solver completed, solution type - optimal mipgap=0.0


([1, 1, 1, 0, 1], -7.079519220008906)

In [40]:
NUMBER_CITIES = 5

Q = np.zeros((NUMBER_CITIES**2, NUMBER_CITIES**2))
w = np.array([
    [0, 15, 17, 16, 16],
    [15, 0, 3, 17, 4],
    [17, 3, 0, 15, 3],
    [16, 17, 15, 0, 12],
    [16, 4, 3, 12, 0]
])

In [41]:
for j in range(NUMBER_CITIES):
    for i in range(NUMBER_CITIES):
        for k in range(NUMBER_CITIES):
            Q[NUMBER_CITIES*i + j, NUMBER_CITIES*k + ((j + 1) % NUMBER_CITIES)] += w[i, k]

In [42]:
fine1 = 20
fine2 = 20

for j in range(NUMBER_CITIES + 1):
    for i in range(NUMBER_CITIES):
        Q[NUMBER_CITIES*i + (j % NUMBER_CITIES), NUMBER_CITIES*i + (j % NUMBER_CITIES)] -= fine1
        for k in range(i + 1, NUMBER_CITIES):
            Q[NUMBER_CITIES*i + (j % NUMBER_CITIES), NUMBER_CITIES*k + (j % NUMBER_CITIES)] += 2*fine1
            
for i in range(NUMBER_CITIES):
    for j in range(NUMBER_CITIES + 1):
        Q[NUMBER_CITIES*i + (j % NUMBER_CITIES), NUMBER_CITIES*i + (j % NUMBER_CITIES)] -= fine2
        for k in range(j + 1, NUMBER_CITIES + 1):
            Q[NUMBER_CITIES*i + (j % NUMBER_CITIES), NUMBER_CITIES*i + (k % NUMBER_CITIES)] += 2*fine2

In [43]:
spins, energy = s.solve_qubo(Q, timeout=30)
spins, energy

[2021-10-23 16:05:48] Solver remote:gurobi started
[2021-10-23 16:05:48] Server status - 0 active tasks
[2021-10-23 16:05:48] Start matrix upload
[2021-10-23 16:05:48] Upload completed, start solving
[2021-10-23 16:05:50] Found solution 0.000000 
[2021-10-23 16:05:50] Found solution -134.000000 
[2021-10-23 16:05:50] Found solution -139.000000 
[2021-10-23 16:05:50] Found solution -151.000000 mipgap=0.7194244604316546
[2021-10-23 16:05:50] Solver completed, solution type - optimal mipgap=0.0


([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
 -151.0)

In [44]:
x = np.array(spins).reshape((NUMBER_CITIES, NUMBER_CITIES))
x

array([[0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0]])

In [45]:
path = []
for j in range(NUMBER_CITIES + 1):
    for i in range(NUMBER_CITIES):
        if x[i, j % NUMBER_CITIES] == 1:
            path.append(str(i + 1))
print(f"\nPath: {'->'.join(path)}")


Path: 5->3->2->1->4->5


In [46]:
from itertools import permutations, product
from copy import copy

min_energy = 10**6
order = None
energies = []
for order_cities in product(range(5), repeat=5):
    if len(set(order_cities)) < 5:
        continue
        
#     if not all(x == y for x, y in zip(order_cities, [4, 2, 1, 0, 3])):
#         continue
    
    x = np.zeros((NUMBER_CITIES, NUMBER_CITIES))
    for step, city in enumerate(order_cities):
        x[city, step] = 1
    x = x.reshape((-1, ))
    
    energies.append(x@Q@x)
    if x@Q@x < min_energy:
        min_energy = x@Q@x
        order = copy(order_cities)
        
min_energy, order

(-151.0, (0, 1, 2, 4, 3))